# How to Experimentos:

- Supondo ambiente CloudLab.us

- Criar a topologia (pode ser utilizando o profile.py - se estiver implementado)

- cada host executa automaticamente o script (caso não tenha executado, fazer manualmente)
` wget -o- https://github.com/NiltonMocelin/FLOWPRI-SDN2/archive/refs/heads/main.zip && unzip main.zip && mv FLOWPRI-SDN2-main FLOWPRI-SDN2 && cd FLOWPRI-SDN2 && sh requirements.sh`

- Assim, todos os requisitos são instalados nos hosts.

- prox passo, ajustar a configuração inicial do controlador e dos switches para cada domínio.

<hr/>

- No cloudlab os ips de cada host são pré-configurados na topologia.

- Nos hosts switch ovs -> adaptar o script ovs_host/script.sh , para as interfaces existentes no host para a topologia definida. E Então, executar o script (demora um pouco, pois ocorre compilação)

- Com isso, os "switches" (que são hosts convencionais executando o processo OVS) estão prontos.

- Dessa forma, sabemos as portas e seus endereços, que dizem qual porta alcança o quê.

- Configurar o arquivo cfg para cada domínio, configurando os switches e rotas (fazemos de modo estático por enquanto) e copiar para FLOWPRI-SDN2/ (pois é de onde o controlador carrega o arquivo)

- os arquivos cfg: observar os ips das interfaces dos switches e quem eles alcançam, para escrever as rotas. Também se atentar ao ip do controlador, o id do switch configurado...

- Assim, os nós devem ser capazes de alcançar um ao outro utilizando o controlador (sh run_flowpri2.sh)

- Cada vez que for realizar um experimento, precisa preencher o cfg do dominio corretamente, pois o nome das interfaces pode alterar (com o profile python acho que poderia fixar, mas não dará tempo para isso)

<hr/>

#### Setup host switch:

- rodar o script em CloudLab/ovs_host

- passar os ips para a interface "switch" criada: ip addr del <ip/mask> dev <interface> e ip addr add <ip/mask> dev <interface>

- modificar as rotas para que os pacotes sejam encaminhados para a interface switch

- (isso tem que rodar pelo script ) ...


#### Problema 1 CloudLab: (resolvido)

- Problema é o seguinte, as rotas estão erradas nos switches - precisa arrumar isso, uma opção é usando "ip netns", a outra opção é organizar manualemente

- Existem várias entrada em "route -n" que ligam um IP para uma interface de saída. Então os pacotes são enviados para a primeira ocorrencia da tabela. Uma solução é criar a rota corretamente usando "sudo ip route add ip-destino dev interface-saida"

- Para isso, precisa-se saber qual interface foi conectada a cada host;;; Por azar, o Cloudlab faz meio randômico. No entanto, na topologia foi descrito o endereço IP que conecta cada interface do switch com o host, então usando alguns truques de bash é possível descobrir qual interface liga quem.

- ISso foi corrigido no ovs_host/requirements_ovs_domX.sh do respectivo domínio ! para facilitar.

-- To achando que não precisa disso, pois depois de configurar o OVS, não vai importar mais.

#### Problema 2 CloudLab: (NAO resolvido)

- CloudLab não está conseguindo criar links entre duas regiões diferentes...


#### Problema 3 CloudLab: (NAO resolvido)

- Os scripts iniciais dos hosts não estão sendo executados automáticamente mais.

- Antes executavam, mas agora, pararam.

- Deve ser simples, vou tentar apagar o comando e criar novamente na topologia... Vai entender o que acontece...


#### Problema 4 CloudLab/Linux em geral: (Resolvido)

- Depois de configurar o ovs, adicionar a bridge, setar link up, adicionar as portas: OK as tabelas openflow funcionam

- Porém, após configurar o controller remoto, o ovs para de funcionar.

- Isso se resolve configurando o fail-mode=secure depois de configurar o controlador ( E SOMENTE DEPOIS, antes vai dar o mesmo problema - talvez funcione antes tbm, mas não testei - ou não lembro)

- Referência: https://docs.openvswitch.org/en/latest/tutorials/ovs-advanced/

- Comando:
```
sudo ovs-vsctl add-br switch \
    -- set bridge switch other-config:datapath-id=0000000000000002 \
    -- add-port switch eth1 -- set interface eth1 ofport_request=1 \
    -- add-port switch eth2 -- set interface eth2 ofport_request=2 \
    -- add-port switch eth3 -- set interface eth3 ofport_request=3 \
    -- add-port switch eth4 -- set interface eth3 ofport_request=4 \
    -- set-controller switch tcp:172.16.2.10:6653
    -- set Bridge switch fail-mode=secure
```

- Explicação: `If we did not do this, then the flow table would start out with a single flow that executes the “normal” action. We could use that feature to yield a switch that behaves the same as the switch we are currently building, but with the caveats described under “Motivation” above.)`


-- CASO ainda de problema -> remover a rota 172.16.X.0/12 eth0 -- pq ai os hosts virtuais que rodam na mesma máquina podem estar se comunicando por loopback por meio do eth0, que é usado pelo CloudLAb exclusivamente. -- > No entanto, pelos experimentos isso não vai acontecer.


# alguns comandos uteis

- Descobrir o numero associado a cada interface, para colocar nas regras de fluxo:  `sudo ovs-vsctl -- --columns=name,ofport list Interface`

- Subir interface: `sudo ifconfig <interface> up`

- Esse comando por inteiro -> não funciona e não retorna erro: 
        ``` sudo ovs-vsctl add-br switch \
         -- set bridge switch other-config:datapath-id=0000000000000001 \
         -- add-port switch eth1 -- set interface eth1 ofport_request=1 \
         -- add-port switch eth2 -- set interface eth2 ofport_request=2 \
         -- add-port switch eth3 -- set interface eth3 ofport_request=3 \
         -- set-controller switch tcp:172.16.1.10:6653 ```

Tem que ser um a um, OBS -- é usado para colocar vários comandos em uma linha.


